In [11]:
import pandas as pd
dados = pd.read_csv('../dados/AIRPOL_data.csv', delimiter=";", header=0)

# Removendo colunas com valores nulos
dados = dados.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
dados

,Country,NUTS_Code,Air_Pollutant,Outcome,Affected_Population,Populated_Area[km2],Air_Pollution_Average[ug/m3],Value
0,Albania,AL,NO2,Asthma,"2337443,00","11299,00","5,50","103,00"
1,Albania,AL,PM2.5,Asthma,"456076,00","11299,00","11,30","231,00"
2,Albania,AL0,NO2,Asthma,"2337443,00","11299,00","5,50","103,00"
3,Albania,AL0,PM2.5,Asthma,"456076,00","11299,00","11,30","231,00"
4,Albania,AL03,PM2.5,Asthma,"149508,00","4041,50","11,30","69,00"
...,...,...,...,...,...,...,...,...
49135,Turkey,TR822,NO2,Stroke,"213615,00","7473,50","9,00","3,00"
49136,Turkey,TRB12,NO2,Stroke,"447268,00","8860,40","12,40","291,00"
49137,Turkey,TRB14,NO2,Stroke,"76591,00","5506,60","12,00","17,00"
49138,Turkey,TRB21,NO2,Stroke,"696344,00","17511,10","11,20","25,00"


In [12]:
import scipy.stats as stats

# Filtrar oa dados para os valores pedidos
dados_filtrados = dados.filter(['Country','NUTS_Code','Air_Pollution_Average[ug/m3]'])
# Restringir a análise aos valores de Portugal
dados_filtrados_portugal = dados_filtrados[dados_filtrados['Country'] == 'Portugal']
dados_filtrados_albania = dados_filtrados[dados_filtrados['Country'] == 'Albania']
dados_filtrados_espanha = dados_filtrados[dados_filtrados['Country'] == 'Spain']
dados_filtrados_franca = dados_filtrados[dados_filtrados['Country'] == 'France']
# Remover as colunas "Country" e "NUTS_Code"
dados_filtrados_portugal = dados_filtrados_portugal.drop(columns=['Country', 'NUTS_Code'])
dados_filtrados_albania = dados_filtrados_albania.drop(columns=['Country', 'NUTS_Code'])
dados_filtrados_espanha = dados_filtrados_espanha.drop(columns=['Country', 'NUTS_Code'])
dados_filtrados_franca = dados_filtrados_franca.drop(columns=['Country', 'NUTS_Code'])

# Retirar uma amostra aleatória de 20 valores de níveis médios de poluição do ar
dados_filtrados_portugal_20 = dados_filtrados_portugal.sample(n=20)
dados_filtrados_albania_20 = dados_filtrados_albania.sample(n=20)
dados_filtrados_espanha_20 = dados_filtrados_espanha.sample(n=20)
dados_filtrados_franca_20 = dados_filtrados_franca.sample(n=20)

dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'].str.replace(',','.').astype(float)
dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'].str.replace(',','.').astype(float)
dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'].str.replace(',','.').astype(float)
dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]'].str.replace(',','.').astype(float)

# Contadores para diferenças significativas, não significativas, normalidade e Levene
diferenca_significativa = 0
diferenca_nao_significativa = 0
normalidade_passou = 0
normalidade_falhou = 0
levene_passou = 0
levene_falhou = 0

# Repetir o processo 100 vezes
for _ in range(1000):
    # Retirar novas amostras aleatórias
    dados_filtrados_portugal_20 = dados_filtrados_portugal.sample(n=20)
    dados_filtrados_albania_20 = dados_filtrados_albania.sample(n=20)
    dados_filtrados_espanha_20 = dados_filtrados_espanha.sample(n=20)
    dados_filtrados_franca_20 = dados_filtrados_franca.sample(n=20)

    # Converter os valores para float
    dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)
    dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)
    dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)
    dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]'] = dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)

    # Teste de Shapiro para determinar a normalidade dos valores
    _, p_value1 = stats.shapiro(dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'])
    _, p_value2 = stats.shapiro(dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'])
    _, p_value3 = stats.shapiro(dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'])
    _, p_value4 = stats.shapiro(dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]'])

    if ((p_value1 < 0.05) | (p_value2 < 0.05) | (p_value3 < 0.05) | (p_value4 < 0.05)):
        normalidade_falhou += 1
        # Teste de Kruskal
        _, p_value = stats.kruskal(
            dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]']
        )
    else:
        normalidade_passou += 1
        # Teste de Levene para igualdade de variâncias
        _, p_value_levene = stats.levene(
            dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'],
            dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]']
        )
        if p_value_levene < 0.05:
            levene_falhou += 1
            # Teste de Kruskal
            _, p_value = stats.kruskal(
                dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]']
            )
        else:
            levene_passou += 1
            # Teste ANOVA
            _, p_value = stats.f_oneway(
                dados_filtrados_portugal_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_albania_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_espanha_20['Air_Pollution_Average[ug/m3]'],
                dados_filtrados_franca_20['Air_Pollution_Average[ug/m3]']
            )

    # Verificar se há diferença significativa
    if p_value < 0.05:
        diferenca_significativa += 1
    else:
        diferenca_nao_significativa += 1

# Criar uma tabela com os resultados
resultados = pd.DataFrame({
    'Resultado': ['Diferença Significativa', 'Não Significativa', 'Normalidade Passou', 'Normalidade Falhou', 'Levene Passou', 'Levene Falhou'],
    'Frequência': [diferenca_significativa, diferenca_nao_significativa, normalidade_passou, normalidade_falhou, levene_passou, levene_falhou]
})

# Exibir a tabela
resultados

,Resultado,Frequência
0,Diferença Significativa,674
1,Não Significativa,326
2,Normalidade Passou,1
3,Normalidade Falhou,999
4,Levene Passou,1
5,Levene Falhou,0
